<a href="https://colab.research.google.com/github/Busola181/Python_From_Zero_To_Hero-/blob/main/AlexNet_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MOUNTING DRIVE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


EXTRACTING ZIP FILE

In [ ]:
# import zipfile

# zip_file_path = '/content/drive/MyDrive/Deep Learning/dogs-vs-cats.zip'
# extract_path = '/content/dogs-vs-cats'

# with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_path)

In [ ]:
# import os
# import shutil
# import random

# path = "/content/train"
# files = os.listdir(path)

# def move_file(init_path, final_path, file_amount):
#     for idx in range(file_amount):
#         random_file = random.choice(files)
#         shutil.move(os.path.join(init_path, random_file), os.path.join(final_path, random_file))
#         files.remove(random_file)


# # move_file('/content/train', '/content/new_dogs_cat/test', 3750)

In [ ]:
# !ls 'drive/MyDrive/Deep Learning/new-dogs-vs-cats/train/cat'

In [ ]:
# def split_data(path):
#     files = os.listdir(path)
#     for file in files:
#         if 'cat' in file:
#             shutil.move(os.path.join(path, file), os.path.join(path, 'cat', file))
#         else:
#             shutil.move(os.path.join(path, file), os.path.join(path, 'dog', file))

# # split_data('drive/MyDrive/Deep Learning/new-dogs-vs-cats/train')
# # split_data('drive/MyDrive/Deep Learning/new-dogs-vs-cats/test')

In [ ]:
# !unzip "/content/dogs-vs-cats/train.zip"

WEIGHT AND BIASES INITIALS

In [ ]:
!pip install --upgrade wandb
import wandb
wandb.login()

wandb: Currently logged in as: oluwabusolaadubuola2023 (oluwabusolaadubuola2023-prometheus). Use `wandb login --relogin` to force relogin


True

DEVICE ASSIGNMENT

In [ ]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


IMPORT LIBRARIES

In [ ]:
import torch
import torch.nn as nn
import torch.functional as F
import torch.optim as optim
from torchvision import transforms as t
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from torchsummary import summary

SCALING DATA

In [ ]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

IMAGE PREPROCESSING

In [ ]:
transform = t.Compose([t.ToTensor(),
                       t.RandomResizedCrop(size = (224,224), scale = (0.1, 0.5)),
                       t.Normalize(mean=mean, std = std ),
                       t.RandomRotation(degrees=15),
                        t.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
                       t.RandomHorizontalFlip(),
                       t.RandomAffine(0, translate = (0.2,0.2), scale =(0.2,0.8),shear = (-10, 10), fill = 255)])



DEFINING EACH DATAPATHS

In [ ]:
training_path = '/content/drive/MyDrive/Deep Learning/new-dogs-vs-cats/train'
testing_path = '/content/drive/MyDrive/Deep Learning/new-dogs-vs-cats/test'
prediction_path = '/content/dogs-vs-cats/dataset/single_prediction'

LOADING THE DATASET

In [ ]:
train_dataset = ImageFolder(root = training_path, transform = transform )
test_dataset = ImageFolder(root = testing_path, transform = transform )
#prediction_dataset = ImageFolder(root = prediction_path, transform = transform )

In [ ]:
BATCH_SIZE = 128

SPLITTING THE DATASET

In [ ]:
train_size = 17500
val_size = 3750
assert train_size + val_size == len(train_dataset)
train_ds, val_ds = random_split(train_dataset,
                                [train_size, val_size])

CREATING DATALOADERS

In [ ]:
train_dl = DataLoader(train_ds, BATCH_SIZE, shuffle = True)
val_dl = DataLoader(val_ds, BATCH_SIZE, shuffle = False)
test_dl = DataLoader(test_dataset, BATCH_SIZE, shuffle = False)

DEFINING THE AlexNet ARCHITECTURE

In [ ]:
class ALEXNETModel(nn.Module):
    def __init__(self):
        super(ALEXNETModel, self).__init__()
        self.features = nn.Sequential(
        nn.Conv2d( 3, 96, kernel_size = (11, 11),stride = 4, padding = 2 ),
        nn.BatchNorm2d(96), nn.ReLU(inplace = True),
        nn.MaxPool2d(kernel_size = (3,3), stride = 2),

        nn.Conv2d(96, 256, kernel_size = (5,5), stride = 1, padding = 2 ),
        nn.BatchNorm2d(256), nn.ReLU(inplace = True),
        nn.MaxPool2d(kernel_size = (3,3), stride = 2),

        nn.Conv2d(256, 384, kernel_size = (3,3), stride = 1, padding = 1),
        nn.BatchNorm2d(384), nn.ReLU(inplace = True),

        nn.Conv2d(384, 384, kernel_size = (3,3), stride = 1,padding = 1),
        nn.BatchNorm2d(384), nn.ReLU(inplace = True),


        nn.Conv2d(384, 256, kernel_size = (3,3), stride = 1, padding = 1),
        nn.BatchNorm2d(256), nn.ReLU(inplace = True),
        nn.MaxPool2d(kernel_size = (3,3), stride = 2)
        )


        self.classifier = nn.Sequential(
            nn.Dropout(p = 0.3),
            nn.Linear(256* 6* 6, 4096), nn.ReLU(inplace = True),
            nn.Dropout(p = 0.4),
            nn.Linear(4096, 4096), nn.ReLU(inplace = True),
            nn.Linear(4096, 1000)
            )


    def forward(self, X):
        out = self.features(X)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

HYPERPARAMETER TUNING

In [ ]:
LEARNING_RATE = 0.001
EPOCHS = 20
BATCH_SIZE = 128
INPUT_SIZE = 154,587
NUM_CLASSES = 1000
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4

MODEL SUMMARY

In [ ]:
model = ALEXNETModel().to(device)
summary(model, input_size = (3, 224, 224) )

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 55, 55]          34,944
       BatchNorm2d-2           [-1, 96, 55, 55]             192
              ReLU-3           [-1, 96, 55, 55]               0
         MaxPool2d-4           [-1, 96, 27, 27]               0
            Conv2d-5          [-1, 256, 27, 27]         614,656
       BatchNorm2d-6          [-1, 256, 27, 27]             512
              ReLU-7          [-1, 256, 27, 27]               0
         MaxPool2d-8          [-1, 256, 13, 13]               0
            Conv2d-9          [-1, 384, 13, 13]         885,120
      BatchNorm2d-10          [-1, 384, 13, 13]             768
             ReLU-11          [-1, 384, 13, 13]               0
           Conv2d-12          [-1, 384, 13, 13]       1,327,488
      BatchNorm2d-13          [-1, 384, 13, 13]             768
             ReLU-14          [-1, 384,

DEFINING LOSS AND OPTIMIZER

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), LEARNING_RATE , MOMENTUM, WEIGHT_DECAY )

EXPERIMENT LOGGING

In [ ]:
run = wandb.init(
    project = "Alex_Net(dogs and cats)",
    config = {"learning_rate ": LEARNING_RATE,
              "epochs": EPOCHS,
              "batch_size": BATCH_SIZE}
)

TRAINING THE MODEL

In [ ]:
def train(model, train_dl, val_dl, epochs, criterion, optimizer, device ):
    wandb.watch(model, log = 'all')
    for epoch in range(EPOCHS):
        model.train()
        epoch_train_loss = 0
        correct_train = 0
        total_train = 0


        for image, labels in train_dl:
            image, labels = image.to(device), labels.to(device)
            optimizer.zero_grad()
            pred = model(image)
            loss = criterion(pred, labels)
            loss.backward()
            optimizer.step()
            epoch_train_loss += loss.item()


            _, predicted = torch.max(pred, 1)
            correct_train += (predicted == labels).sum().item()
            total_train += labels.size(0)

        avg_train_loss = epoch_train_loss / len(train_dl)
        train_accuracy = 100 * correct_train / total_train


        model.eval()
        epoch_val_loss = 0
        correct_val = 0
        total_val = 0
        with torch.no_grad():
            for image, labels in val_dl:
                image, labels = image.to(device), labels.to(device)
                pred = model(image)
                loss = criterion(pred, labels)
                epoch_val_loss += loss.item()

                _, predicted = torch.max(pred, 1)
                correct_val += (predicted == labels).sum().item()
                total_val += labels.size(0)


        avg_val_loss = epoch_val_loss/ len(val_dl)
        val_accuracy = 100 * correct_val / total_val


        if (epoch + 1) % 1 == 0:
                print(f'Epoch {epoch + 1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.2f}%, '
                      f'Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.2f}%')



        wandb.log({
                 "Train Loss": avg_train_loss,
                 "Train Accuracy": train_accuracy,
                 "Validation Loss": avg_val_loss,
                 "Validation Accuracy": val_accuracy,
             })


IMPLEMENTING THE TRAIN FUNCTION

In [ ]:
train(model, train_dl, val_dl, EPOCHS, criterion, optimizer, device)

TESTING THE MODEL

In [ ]:
def test(model, test_dl, criterion, device):
     with torch.no_grad():
        model.eval()
        test_loss = 0
        correct = 0
        total = 0

        # wandb.init()

        for image, labels in test_dl:
            image, labels = image.to(device), labels.to(device)
            pred = model(image)
            loss = criterion(pred, labels)
            test_loss += loss.item()

            _, predicted = torch.max(pred, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        avg_test_loss = test_loss / len(test_dl)
        test_accuracy = 100 * correct / total

        print(f'Test Loss: {avg_test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

        # wandb.log({
        #     "Test Loss": avg_test_loss,
        #     "Test Accuracy": test_accuracy
        # })

        # wandb.finish()

        return avg_test_loss, test_accuracy


IMPLEMENTING THE TEST FUNCTION

In [ ]:
test(model, test_dl, criterion, device)

In [ ]:
network_checkpoint = {'model': ALEXNETModel(),
                      'state_dict': model.state_dict(),
                      'optimizer_dict': optimizer.state_dict(),
                      'No_of_epochs': EPOCHS}
torch.save(network_checkpoint, 'network_checkpoint.pth')


In [ ]:
torch.load('network_checkpoint.pth')